In [64]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [66]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 3, 3])


In [67]:
input = torch.randn(1, 1, 32, 32)
output = net(input)
print(output)

tensor([[ 0.0404, -0.0109, -0.1105,  0.0938, -0.1112,  0.0053, -0.1165, -0.0038,
          0.0023, -0.1005]], grad_fn=<AddmmBackward>)


In [68]:
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.6440, grad_fn=<MseLossBackward>)


In [69]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [70]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
None
conv1.bias.grad after backward
tensor([ 0.0029, -0.0021, -0.0193, -0.0034, -0.0010,  0.0176])


In [71]:
x = net.conv1.bias.detach()
x_grad = net.conv1.bias.grad.detach()
print('Before weight update: ', x.data)
print('Gradients for weight: ', x_grad.data)

# Stochastic gradient descent: w = w - lr * wgrad
learning_rate = 0.01
x_new = x - (x_grad * learning_rate)
print('After weight update : ', x_new.data)

# learning_rate = 0.01
# x.data.sub_(x.grad.data * learning_rate)

Before weight update:  tensor([ 0.2962, -0.0497, -0.0947, -0.1363,  0.1877,  0.0358])
Gradients for weight:  tensor([ 0.0029, -0.0021, -0.0193, -0.0034, -0.0010,  0.0176])
After weight update :  tensor([ 0.2962, -0.0497, -0.0945, -0.1363,  0.1877,  0.0356])


In [72]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update

print('After optim step : ', net.conv1.bias.data)

After optim step :  tensor([ 0.2962, -0.0497, -0.0945, -0.1363,  0.1877,  0.0356])
